개미는.. 뚠뚠.. 오늘도.. 뚠뚠.. 데이터를.. 까본다네.. ^ㅜ

# import packages

In [1]:
import pandas as pd
import numpy as np

import requests
import zipfile
import io

# Data Load
: google colab 환경에서 data 불러오기

In [ ]:
# 구글 드라이브 공유 링크에서 파일 ID를 가져옵니다.
file_id = '1ZUueEhjvhzmo8UpZLwqMgzMKA6FGtLJk'  # 여기에 파일 ID를 입력하세요.
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# 파일 다운로드
response = requests.get(download_url)
response.raise_for_status()  # 요청이 실패하면 예외를 발생시킵니다.

# 다운로드한 ZIP 파일을 메모리에서 직접 압축 해제합니다.
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall('/content/unzipped_folder')

# 결과 확인
import os
os.listdir('/content/unzipped_folder')


In [39]:
# 데이터 불러오기
df = pd.read_csv("/content/unzipped_folder/train.csv")
train = pd.read_csv("/content/unzipped_folder/train.csv")
test = pd.read_csv("/content/unzipped_folder/test.csv")
submission = pd.read_csv("/content/unzipped_folder/sample_submission.csv")

In [ ]:
print(train)

In [ ]:
print(test)

# Data Encoding

## 1 Binary Encoding
: WT = 0, Variants = 1


### data load

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### data preprocessing
- WT = 0
- Variants = 1

#### 기존 방식

In [ ]:
# WT는 0, 변이는 1로 변환하는 함수
train_binary = train.replace('WT', 0)
train_binary.iloc[:, 2:] = train_binary.iloc[:, 2:].applymap(lambda x: 1 if x != 0 else 0)
print(train_binary)

#### 새로운 방식

In [2]:
# NaN 값을 'WT'로 치환
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

train_nan = train.isna().sum().sum()
print(f"처리 후 train NaN 개수: {train_nan}")

test_nan = test.isna().sum().sum()
print(f"처리 후 test NaN 개수: {test_nan}")

처리 후 train NaN 개수: 0
처리 후 test NaN 개수: 0


In [3]:
import re
def process_value_be(value):
    # 0. WT 그대로 유지
    if value == 'WT':
        return 0

    # 1. 띄어쓰기 기준으로 변이 파트 분리
    parts = value.split()

    for part in parts:
        # 1.1 WT: 0
        if part == 'WT':
            continue

        # 1.2 알파벳 + 숫자 + 알파벳
        elif re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part):
            match = re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part)
            prefix = match.group(1)   # 숫자 앞의 문자 그룹
            suffix = match.group(2)   # 숫자 뒤의 문자 그룹

            # 1.2.1 숫자 앞뒤의 문자 그룹이 같으면 silent
            if prefix.isupper() and suffix.isupper() and prefix == suffix:
                continue

            # 1.2.2 앞뒤 문자 그룹이 다르면 missense
            elif prefix.isupper() and suffix.isupper() and prefix != suffix:
                return 1

            # 1.2.3 뒤의 문자 그룹에 '*'이 들어가면 nonsense
            elif '*' in suffix:
                return 1

            # 1.2.4 뒤의 문자 그룹에 'fs'가 들어가면 frameshift
            elif 'fs' in suffix:
                return 1

            # 1.2.5 뒤에 문자 그룹에 'del'이나 'ins'이 들어가면 indel
            elif 'del' in suffix or 'ins' in suffix or not(suffix):
                return 1

            # 1.2.7 예외사항 출력
            else:
                print(part)

        # 1.3 숫자_숫자알파벳>알파벳
        elif re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part):
            match = re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part)
            prenum = int(match.group(1))  # 앞 위치
            postnum = int(match.group(2))  # 뒷 위치
            variant = match.group(3)  # 변이 정보 그룹

            # 1.3.1 >가 있는 경우
            if '>' in variant:
                before, after = variant.split('>')  # '>' 기준으로 나눔

                # 1.3.1.1 문자 그룹이 같은 경우 -> silent
                if before.isupper() and after.isupper() and before == after:
                    continue

                # 1.3.1.2 *가 있는 경우 -> nonsense
                elif '*' in after:
                    return 1

                # 1.3.1.3 문자 그룹이 다른 경우 -> missense
                elif before.isupper() and after.isupper() and before != after:
                    return 1

                else:
                    print(part)

            # 1.3.2 > 대신 'del'이 있는 경우 -> indel
            elif 'del' in variant:
                return 1

            # 1.3.3 예외사항 출력
            else :
                print(part)

        # 1.4 알파벳숫자_알파벳숫자변이정보
        elif re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part):
            match = re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part)
            prenum = int(match.group(2))   # 앞 위치
            postnum = int(match.group(4))  # 뒷 위치
            variant = match.group(5)  # 변이정보

            # 1.4.1 'delins'가 있는 경우 -> indel
            if 'del' in variant or 'ins' in variant:
                return 1

            # 1.4.4 예외사항 출력
            else :
                print(part)

    return 0

In [4]:
# train 데이터프레임 전처리
for col in train.columns[2:]:  # ID와 SUBCLASS 제외
    train[col] = train[col].apply(process_value_be)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(train.iloc[:, 2:].values)
print("train 유니크한 값: ")
print(unique_values)

# test 데이터프레임 전처리
for col in test.columns[1:]:  # ID 제외
    test[col] = test[col].apply(process_value_be)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(test.iloc[:, 2:].values)
print("test 유니크한 값: ")
print(unique_values)

train 유니크한 값: 
[0 1]
test 유니크한 값: 
[0 1]


In [5]:
print(train)
print(test)

              ID SUBCLASS  A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  \
0     TRAIN_0000    KIPAN    0     0      0      0     0      0      0      0   
1     TRAIN_0001     SARC    0     0      0      0     0      0      0      0   
2     TRAIN_0002     SKCM    0     0      0      0     0      0      0      0   
3     TRAIN_0003     KIRC    0     0      0      0     0      0      0      0   
4     TRAIN_0004   GBMLGG    0     0      0      0     0      0      0      0   
...          ...      ...  ...   ...    ...    ...   ...    ...    ...    ...   
6196  TRAIN_6196     LUAD    0     0      0      0     0      0      0      0   
6197  TRAIN_6197      LGG    0     0      0      0     0      0      0      0   
6198  TRAIN_6198     COAD    0     0      0      0     0      0      0      0   
6199  TRAIN_6199     TGCT    0     0      0      0     0      0      0      0   
6200  TRAIN_6200     SKCM    0     0      0      0     0      0      0      0   

      ...  ZNF292  ZNF365  

In [6]:
train.to_csv('data/train_be.csv', index=False)
test.to_csv('data/test_be.csv', index=False)

### importance feature

In [ ]:
# 데이터 분리
X = train.drop(columns=['ID', 'SUBCLASS'])
y = train['SUBCLASS']

# Label Encoding for SUBCLASS (암 유형)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced_subsample')
rf.fit(X_train, y_train)

# 중요도 출력
importances = rf.feature_importances_
importance_df = pd.DataFrame({'Gene': X.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 상위 n개 유전자 출력
print(importance_df.head(10))

In [ ]:
# 중요도가 높은 상위 n개의 유전자 리스트
top_genes = importance_df['Gene'].head(30).tolist()
print(top_genes)

# 상위 n개의 유전자만 추출
X_top = X[top_genes]

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_top, y_encoded, test_size=0.2, random_state=42)

# 모든 열을 숫자형으로 변환 (바이너리 인코딩 후)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')

In [ ]:
# 기존 리스트들
original = ['BRAF', 'IDH1', 'VHL', 'PIK3CA', 'TP53', 'APC', 'PTEN', 'NPM1', 'ATRX', 
            'CTNNB1', 'GATA3', 'CDH1', 'EGFR', 'PCLO', 'SPOP', 'MAP3K1', 'CDKN2A', 
            'RYR2', 'KMT2D', 'SYNE1', 'SPTA1', 'KIT', 'NF1', 'MXRA5', 'NOTCH1', 
            'IDH2', 'RYR1', 'PABPC1', 'DST', 'HRAS']

balanced = ['BRAF', 'IDH1', 'TP53', 'VHL', 'APC', 'NPM1', 'PIK3CA', 'PTEN', 'ATRX', 
            'CTNNB1', 'CDKN2A', 'PCLO', 'KMT2D', 'LRIG1', 'RYR2', 'SPOP', 'EGFR', 
            'KIT', 'SYNE1', 'ERCC2', 'SPTA1', 'IDH2', 'NF1', 'HRAS', 'FBXW7', 'NOTCH1', 
            'GATA3', 'CDH1', 'CEBPA', 'DMD']

balanced_subsample = ['IDH1', 'BRAF', 'TP53', 'VHL', 'APC', 'NPM1', 'PIK3CA', 'PTEN', 'CTNNB1', 
                      'ATRX', 'CDKN2A', 'PCLO', 'RYR2', 'SPOP', 'KMT2D', 'KIT', 'LRIG1', 'SYNE1', 
                      'EGFR', 'ERCC2', 'SPTA1', 'IDH2', 'NF1', 'HRAS', 'FBXW7', 'CEBPA', 'GATA3', 
                      'NOTCH1', 'DMD', 'CDH1']

# 세 리스트의 합집합 구하기
top_genes = list(set(original) | set(balanced) | set(balanced_subsample))

# 결과 출력
print(sorted(top_genes))

# 상위 n개의 유전자만 추출
X_top = X[top_genes]

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_top, y_encoded, test_size=0.2, random_state=42)

# 모든 열을 숫자형으로 변환 (바이너리 인코딩 후)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')

### RF, XGB, LGBM train

In [ ]:
classes = np.unique(y_train)
class_counts = np.bincount(y_train)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))

# 클래스 가중치 딕셔너리 출력
print("Class Weights Dictionary:", class_weight_dict)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
rf.fit(X_train, y_train)

# 예측 및 평가
y_pred = rf.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight

#"""balanced
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
#"""

"""sqrtbalanced
classes, class_counts = np.unique(y_train, return_counts=True)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
"""


# XGBoost 모델 학습
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# 예측 및 평가
y_pred = xgb.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# LightGBM 모델 학습
lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, min_gain_to_split=0.01, class_weight=class_weight_dict)
lgbm.fit(X_train, y_train)

# 예측 및 평가
y_pred = lgbm.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"LightGBM Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

### test predict

In [ ]:
# WT는 0, 변이는 1로 변환하는 함수
test_binary = test.replace('WT', 0)
test_binary.iloc[:, 1:] = test_binary.iloc[:, 1:].applymap(lambda x: 1 if x != 0 else 0)
print(test_binary)

In [ ]:
X_test = test.drop(columns=['ID'])

# 상위 n개의 유전자만 추출
X_test_filtered = X_test[top_genes]
X_test_filtered = X_test_filtered.apply(pd.to_numeric, errors='coerce')
print(X_test_filtered)

In [ ]:
# 예측 및 평가
y_pred_rf = rf.predict(X_test_filtered)
y_pred_xgb = xgb.predict(X_test_filtered)
y_pred_lgbm = lgbm.predict(X_test_filtered)

In [ ]:
submission = pd.read_csv('open/sample_submission.csv')
print("predictions의 길이:", len(y_pred_rf), len(y_pred_xgb), len(y_pred_lgbm))
print("submisson의 길이:", len(submission))

In [43]:
predictions_rf = y_pred_rf.ravel()  # 또는 predictions.flatten()
predictions_rf = le.inverse_transform(predictions_rf)
submission["SUBCLASS"] = predictions_rf
submission.to_csv('submission_rf_.csv', encoding='UTF-8-sig', index=False)

predictions_xgb = y_pred_xgb.ravel()  # 또는 predictions.flatten()
predictions_xgb = le.inverse_transform(predictions_xgb)
submission["SUBCLASS"] = predictions_xgb
submission.to_csv('submission_xgb_.csv', encoding='UTF-8-sig', index=False)

predictions_lgbm = y_pred_lgbm.ravel()  # 또는 predictions.flatten()
predictions_lgbm = le.inverse_transform(predictions_lgbm)
submission["SUBCLASS"] = predictions_lgbm
submission.to_csv('submission_lgbm_.csv', encoding='UTF-8-sig', index=False)

### result
**바이너리 인코딩**: 변이가 존재하면 1, 없으면 0으로 인코딩.

→ 상위 30개로 RF(**0.21816**), XGB(**0.22777**), LGBM(**0.23187**)

## 2 One-Hot Encoding
wt, silent, missense, indel, frameshift, nonsense

### data load

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')

### data preprocessing

In [ ]:
# NaN 값을 'WT'로 치환
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

train_nan = train.isna().sum().sum()
print(f"처리 후 train NaN 개수: {train_nan}")

test_nan = test.isna().sum().sum()
print(f"처리 후 test NaN 개수: {test_nan}")

In [3]:
def process_value(value):
    # 0. WT 그대로 유지
    if value == 'WT':
        return [1, 0, 0, 0, 0, 0]

    # 1. 띄어쓰기 기준으로 변이 파트 분리
    parts = value.split()

    # WT, silent, missense, indel, frameshift, nonsense
    processed_parts = [0, 0, 0, 0, 0, 0]

    for part in parts:
        # 1.1 WT: 0
        if part == 'WT':
            processed_parts[0] = 1
            continue

        # 1.2 알파벳 + 숫자 + 알파벳
        elif re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part):
            match = re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part)
            prefix = match.group(1)   # 숫자 앞의 문자 그룹
            suffix = match.group(2)   # 숫자 뒤의 문자 그룹

            # 1.2.1 숫자 앞뒤의 문자 그룹이 같으면 silent
            if prefix.isupper() and suffix.isupper() and prefix == suffix:
                processed_parts[1] = 1

            # 1.2.2 앞뒤 문자 그룹이 다르면 missense
            elif prefix.isupper() and suffix.isupper() and prefix != suffix:
                processed_parts[2] = 1

            # 1.2.3 뒤의 문자 그룹에 '*'이 들어가면 nonsense
            elif '*' in suffix:
                processed_parts[5] = 1

            # 1.2.4 뒤의 문자 그룹에 'fs'가 들어가면 frameshift
            elif 'fs' in suffix:
                processed_parts[4] = 1

            # 1.2.5 뒤에 문자 그룹에 'del'이나 'ins'이 들어가면 indel
            elif 'del' in suffix or 'ins' in suffix or not(suffix):
                processed_parts[3] = 1

            # 1.2.7 예외사항 출력
            else:
                print(part)

        # 1.3 숫자_숫자알파벳>알파벳
        elif re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part):
            match = re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part)
            prenum = int(match.group(1))  # 앞 위치
            postnum = int(match.group(2))  # 뒷 위치
            variant = match.group(3)  # 변이 정보 그룹

            # 1.3.1 >가 있는 경우
            if '>' in variant:
                before, after = variant.split('>')  # '>' 기준으로 나눔

                # 1.3.1.1 문자 그룹이 같은 경우 -> silent
                if before.isupper() and after.isupper() and before == after:
                    processed_parts[1] = 1

                # 1.3.1.2 *가 있는 경우 -> nonsense
                elif '*' in after:
                    processed_parts[5] = 1

                # 1.3.1.3 문자 그룹이 다른 경우 -> missense
                elif before.isupper() and after.isupper() and before != after:
                    processed_parts[2] = 1

                else:
                    print(part)

            # 1.3.2 > 대신 'del'이 있는 경우 -> indel
            elif 'del' in variant:
                processed_parts[3] = 1

            # 1.3.3 예외사항 출력
            else :
                print(part)

        # 1.4 알파벳숫자_알파벳숫자변이정보
        elif re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part):
            match = re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part)
            prenum = int(match.group(2))   # 앞 위치
            postnum = int(match.group(4))  # 뒷 위치
            variant = match.group(5)  # 변이정보

            # 1.4.1 'delins'가 있는 경우 -> indel
            if 'del' in variant or 'ins' in variant:
                processed_parts[3] = 1

            # 1.4.4 예외사항 출력
            else :
                print(part)

    return processed_parts

In [ ]:
# train 데이터프레임 전처리
for col in train.columns[2:]:  # ID와 SUBCLASS 제외
    train[col] = train[col].apply(process_value)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(train.iloc[:, 2:].values)
print(unique_values)

In [ ]:
# test 데이터프레임 전처리
for col in test.columns[1:]:  # ID 제외
    test[col] = test[col].apply(process_value)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(test.iloc[:, 1:].values)
print(unique_values)

#### 파일 저장

In [6]:
train.to_csv('train_oh.csv', index=False)
test.to_csv('test_oh.csv',  index=False)

### importance feature

In [ ]:
X = test.drop(columns=['ID'])

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(X.values)
print(unique_values)

In [ ]:
# 현재 시간을 출력하는 함수
def print_with_time(*messages):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message = ' '.join(map(str, messages))  # 튜플 대신 문자열로 변환
    print(f"[{current_time}] {message}")

# Random Forest 모델 설정
def compute_feature_importance(X_part, y):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_part, y)
    importances = rf.feature_importances_
    return pd.DataFrame({'Feature': X_part.columns, 'Importance': importances})

# 데이터 분할
def split_features(X, step):
    for i in range(0, X.shape[1], step):
        yield i, X.iloc[:, i:i + step]

def calculate_total_importance(X, y, step):
    importance_list = []
    for i, X_part in split_features(X, step):
        print_with_time(f"{i//step + 1}/22 part start")
        
        # 각 리스트를 개별 열로 확장
        X_expanded = pd.concat([X_part[col].apply(pd.Series).add_prefix(f'{col}_') for col in X_part.columns], axis=1)
        print_with_time(f"{i//step + 1}/22 part expanded done")
        importance_df = compute_feature_importance(X_expanded, y)
        print_with_time(f"{i//step + 1}/22 part importance done")
        importance_list.append(importance_df)
        print_with_time(f"{i//step + 1}/22 part done")
    
    return pd.concat(importance_list).sort_values(by='Importance', ascending=False)

# y는 SUBCLASS, X는 4384개의 피처
X = train.drop(columns=['ID', 'SUBCLASS'])
y = train['SUBCLASS']

# Label Encoding for SUBCLASS (암 유형)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

importance_result = calculate_total_importance(X, y_encoded, step=200)

# 상위 20개 특성 출력
print(importance_result.head(20))

# importance_result를 파일로 저장하는 코드 추가
importance_result.to_csv('importance_result.csv', index=False)

In [ ]:
importance_result = pd.read_csv('importance_result.csv')
top_500 = importance_result.head(500)
gene_names = top_500['Feature'].apply(lambda x: x.split('_')[0]).unique()

print("important genes: ", gene_names)
print("genes num: ", len(gene_names))

### RF, XGB, LGBM train

In [21]:
X = train[gene_names]
X_expanded = pd.concat([X[col].apply(pd.Series).add_prefix(f'{col}_') for col in X.columns], axis=1)

In [ ]:
print(X_expanded)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X_expanded, y_encoded, test_size=0.2, random_state=42)

In [ ]:
classes = np.unique(y_train)
class_counts = np.bincount(y_train)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))

# 클래스 가중치 딕셔너리 출력
print("Class Weights Dictionary:", class_weight_dict)

In [ ]:
# 클래스 가중치를 0.5 단위로 반올림
rounded_class_weight_dict = {k: round(v * 2) / 2 for k, v in class_weight_dict.items()}

# 클래스 가중치 딕셔너리 출력
print("Rounded Class Weights Dictionary:", rounded_class_weight_dict)

In [ ]:
classes_to_increase = [24]
for cls in classes_to_increase:
    if cls in rounded_class_weight_dict:
        rounded_class_weight_dict[cls] += 0.10

# 클래스 가중치 딕셔너리 출력
print("Updated Class Weights Dictionary:", rounded_class_weight_dict)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=rounded_class_weight_dict)
rf.fit(X_train, y_train)

# 예측 및 평가
y_pred = rf.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight

""" balanced
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
"""

#"""sqrtbalanced
classes, class_counts = np.unique(y_train, return_counts=True)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
#"""

# XGBoost 모델 학습
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# 예측 및 평가
y_pred = xgb.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
import lightgbm as lgb

# LightGBM 모델 학습
lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, min_gain_to_split=0.01, class_weight=rounded_class_weight_dict)
lgbm.fit(X_train, y_train)

# 예측 및 평가
y_pred = lgbm.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"LightGBM Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

### test predict

In [ ]:
X_test = test[gene_names]
X_test_expanded = pd.concat([X_test[col].apply(pd.Series).add_prefix(f'{col}_') for col in X_test.columns], axis=1)

print(X_test_expanded)

In [ ]:
# 예측 및 평가
y_pred_rf = rf.predict(X_test_expanded)
y_pred_xgb = xgb.predict(X_test_expanded)
y_pred_lgbm = lgbm.predict(X_test_expanded)

submission = pd.read_csv('open/sample_submission.csv')
print("predictions의 길이:", len(y_pred_rf), len(y_pred_xgb), len(y_pred_lgbm))
print("submisson의 길이:", len(submission))

In [29]:
predictions_rf = y_pred_rf.ravel()  # 또는 predictions.flatten()
predictions_rf = le.inverse_transform(predictions_rf)
submission["SUBCLASS"] = predictions_rf
submission.to_csv('submission_rf_oh.csv', encoding='UTF-8-sig', index=False)

predictions_xgb = y_pred_xgb.ravel()  # 또는 predictions.flatten()
predictions_xgb = le.inverse_transform(predictions_xgb)
submission["SUBCLASS"] = predictions_xgb
submission.to_csv('submission_xgb_oh.csv', encoding='UTF-8-sig', index=False)

predictions_lgbm = y_pred_lgbm.ravel()  # 또는 predictions.flatten()
predictions_lgbm = le.inverse_transform(predictions_lgbm)
submission["SUBCLASS"] = predictions_lgbm
submission.to_csv('submission_lgbm_oh.csv', encoding='UTF-8-sig', index=False)

### result
**원핫 인코딩**: WT = _0, silent = _1, missens = _2, indel = _3, frameshift = _4, nonsense = _5

→ 상위 30개로 RF(**0.19896**), XGB(**0.20651**), LGBM(**0.20746**)

## 3 label Encoding
- wt, silent -> 1
- missense -> 2
- indel -> 3
- frameshift -> 4
- nonsense -> 5

### data load

In [7]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
train = pd.read_csv('train_le.csv')
test = pd.read_csv('test_le.csv')

In [ ]:
print(train)
print(test)

### data preprocessing

In [ ]:
# NaN 값을 'WT'로 치환
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

train_nan = train.isna().sum().sum()
print(f"처리 후 train NaN 개수: {train_nan}")

test_nan = test.isna().sum().sum()
print(f"처리 후 test NaN 개수: {test_nan}")

In [3]:
def process_value_le(value):
    # 0. WT 그대로 유지
    if value == 'WT':
        return 1

    # 1. 띄어쓰기 기준으로 변이 파트 분리
    parts = value.split()

    # WT, silent, missense, indel, frameshift, nonsense
    processed_value = 0

    for part in parts:
        # 1.1 WT: 0
        if part == 'WT':
            processed_value = max(processed_value, 1)
            continue

        # 1.2 알파벳 + 숫자 + 알파벳
        elif re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part):
            match = re.match(r'^([A-Za-z*-]*)\d+([A-Za-z*-]*)$', part)
            prefix = match.group(1)   # 숫자 앞의 문자 그룹
            suffix = match.group(2)   # 숫자 뒤의 문자 그룹

            # 1.2.1 숫자 앞뒤의 문자 그룹이 같으면 silent
            if prefix.isupper() and suffix.isupper() and prefix == suffix:
                processed_value = max(processed_value, 1)

            # 1.2.2 앞뒤 문자 그룹이 다르면 missense
            elif prefix.isupper() and suffix.isupper() and prefix != suffix:
                processed_value = max(processed_value, 2)

            # 1.2.3 뒤의 문자 그룹에 '*'이 들어가면 nonsense
            elif '*' in suffix:
                processed_value = max(processed_value, 5)

            # 1.2.4 뒤의 문자 그룹에 'fs'가 들어가면 frameshift
            elif 'fs' in suffix:
                processed_value = max(processed_value, 4)

            # 1.2.5 뒤에 문자 그룹에 'del'이나 'ins'이 들어가면 indel
            elif 'del' in suffix or 'ins' in suffix or not(suffix):
                processed_value = max(processed_value, 3)

            # 1.2.7 예외사항 출력
            else:
                print(part)

        # 1.3 숫자_숫자알파벳>알파벳
        elif re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part):
            match = re.match(r'^(\d+)_(\d+)([A-Za-z*-]+>[A-Za-z*-]+|del)$', part)
            prenum = int(match.group(1))  # 앞 위치
            postnum = int(match.group(2))  # 뒷 위치
            variant = match.group(3)  # 변이 정보 그룹

            # 1.3.1 >가 있는 경우
            if '>' in variant:
                before, after = variant.split('>')  # '>' 기준으로 나눔

                # 1.3.1.1 문자 그룹이 같은 경우 -> silent
                if before.isupper() and after.isupper() and before == after:
                    processed_value = max(processed_value, 1)

                # 1.3.1.2 *가 있는 경우 -> nonsense
                elif '*' in after:
                    processed_value = max(processed_value, 5)

                # 1.3.1.3 문자 그룹이 다른 경우 -> missense
                elif before.isupper() and after.isupper() and before != after:
                    processed_value = max(processed_value, 2)

                else:
                    print(part)

            # 1.3.2 > 대신 'del'이 있는 경우 -> indel
            elif 'del' in variant:
                processed_value = max(processed_value, 3)

            # 1.3.3 예외사항 출력
            else :
                print(part)

        # 1.4 알파벳숫자_알파벳숫자변이정보
        elif re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part):
            match = re.match(r'^([A-Za-z]+)(\d+)_([A-Za-z]+)?(\d+)([A-Za-z]+)$', part)
            prenum = int(match.group(2))   # 앞 위치
            postnum = int(match.group(4))  # 뒷 위치
            variant = match.group(5)  # 변이정보

            # 1.4.1 'delins'가 있는 경우 -> indel
            if 'del' in variant or 'ins' in variant:
                processed_value = max(processed_value, 3)

            # 1.4.4 예외사항 출력
            else :
                print(part)

    return processed_value

In [ ]:
# train 데이터프레임 전처리
for col in train.columns[2:]:  # ID와 SUBCLASS 제외
    train[col] = train[col].apply(process_value_le)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(train.iloc[:, 2:].values)
print(unique_values)

In [ ]:
# test 데이터프레임 전처리
for col in test.columns[1:]:  # ID 제외
    test[col] = test[col].apply(process_value_le)

# 모든 열에서 유니크한 값 추출
unique_values = np.unique(test.iloc[:, 1:].values)
print(unique_values)

#### 파일 저장

In [6]:
train.to_csv('train_le.csv', index=False)
test.to_csv('test_le.csv',  index=False)

### importance feature

In [ ]:
# 데이터 분리
X = train.drop(columns=['ID', 'SUBCLASS'])
y = train['SUBCLASS']

# Label Encoding for SUBCLASS (암 유형)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced_subsample')
rf.fit(X_train, y_train)

# 중요도 출력
importances = rf.feature_importances_
importance_df = pd.DataFrame({'Gene': X.columns, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 상위 n개 유전자 출력
print(importance_df.head(30))

In [ ]:
# 중요도가 높은 상위 n개의 유전자 리스트
top_genes = importance_df['Gene'].head(30).tolist()
print(top_genes)

In [ ]:
normal = ['BRAF', 'IDH1', 'VHL', 'APC', 'TP53', 'PIK3CA', 'ATRX', 'GATA3', 'PTEN', 'NPM1', 'CDH1', 'CTNNB1', 'EGFR', 'MAP3K1', 'CDKN2A', 'PCLO', 'KMT2D', 'SPOP', 'RYR2', 'SYNE1', 'NF1', 'SPTA1', 'KIT', 'NOTCH1', 'MXRA5', 'IDH2', 'RB1', 'FBXW7', 'DST', 'RYR1']
balanced = ['BRAF', 'IDH1', 'APC', 'TP53', 'VHL', 'NPM1', 'PTEN', 'PIK3CA', 'ATRX', 'CTNNB1', 'KMT2D', 'CDKN2A', 'PCLO', 'RYR2', 'LRIG1', 'NF1', 'SPOP', 'SYNE1', 'ERCC2', 'KIT', 'EGFR', 'GATA3', 'CDH1', 'SPTA1', 'FBXW7', 'IDH2', 'NOTCH1', 'HRAS', 'RB1', 'CEBPA']
balanced_subsample = ['IDH1', 'BRAF', 'APC', 'TP53', 'VHL', 'NPM1', 'PTEN', 'PIK3CA', 'ATRX', 'CTNNB1', 'KMT2D', 'CDKN2A', 'RYR2', 'PCLO', 'NF1', 'SPOP', 'LRIG1', 'KIT', 'EGFR', 'SYNE1', 'GATA3', 'CDH1', 'ERCC2', 'SPTA1', 'RB1', 'FBXW7', 'HRAS', 'IDH2', 'DMD', 'CEBPA']

# 세 리스트의 합집합 구하기
top_genes = set(normal) | set(balanced) | set(balanced_subsample)

# 합집합을 리스트로 변환
top_genes = list(top_genes)

# 결과 출력
print("Union of the three lists:", top_genes)
print("Number of genes:", len(top_genes))

### RF, XGB, LGBM train

In [17]:
# 상위 n개의 유전자만 추출
X_top = X[top_genes]

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_top, y_encoded, test_size=0.2, random_state=42)

In [ ]:
classes = np.unique(y_train)
class_counts = np.bincount(y_train)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))

# 클래스 가중치 딕셔너리 출력
print("Class Weights Dictionary:", class_weight_dict)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
rf.fit(X_train, y_train)

# 예측 및 평가
y_pred = rf.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight

""" balanced
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
"""

#"""sqrtbalanced
classes, class_counts = np.unique(y_train, return_counts=True)
sqrt_class_weights = np.sqrt(class_counts.sum() / class_counts)
class_weight_dict = dict(zip(classes, sqrt_class_weights))
sample_weights = np.array([class_weight_dict[label] for label in y_train])
#"""

# XGBoost 모델 학습
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# 예측 및 평가
y_pred = xgb.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"XGBoost Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
import lightgbm as lgb

# LightGBM 모델 학습
lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42, min_gain_to_split=0.01, class_weight=class_weight_dict)
lgbm.fit(X_train, y_train)

# 예측 및 평가
y_pred = lgbm.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"LightGBM Accuracy: {accuracy:.4f}")
print()

# 혼동 행렬 계산 및 출력
conf_matrix = confusion_matrix(y_val, y_pred)
# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')  # 이미지 파일로 저장
plt.show()

# 매크로 F1 스코어 계산 및 출력
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f"Macro F1 Score: {macro_f1:.4f}")

### test predict

In [ ]:
X_test = test[gene_names]
X_test_expanded = pd.concat([X_test[col].apply(pd.Series).add_prefix(f'{col}_') for col in X_test.columns], axis=1)

print(X_test_expanded)

In [ ]:
# 예측 및 평가
y_pred_rf = rf.predict(X_test_expanded)
y_pred_xgb = xgb.predict(X_test_expanded)
y_pred_lgbm = lgbm.predict(X_test_expanded)

submission = pd.read_csv('open/sample_submission.csv')
print("predictions의 길이:", len(y_pred_rf), len(y_pred_xgb), len(y_pred_lgbm))
print("submisson의 길이:", len(submission))

In [ ]:
predictions_rf = y_pred_rf.ravel()  # 또는 predictions.flatten()
predictions_rf = le.inverse_transform(predictions_rf)
submission["SUBCLASS"] = predictions_rf
submission.to_csv('submission_rf_oh.csv', encoding='UTF-8-sig', index=False)

predictions_xgb = y_pred_xgb.ravel()  # 또는 predictions.flatten()
predictions_xgb = le.inverse_transform(predictions_xgb)
submission["SUBCLASS"] = predictions_xgb
submission.to_csv('submission_xgb_oh.csv', encoding='UTF-8-sig', index=False)

predictions_lgbm = y_pred_lgbm.ravel()  # 또는 predictions.flatten()
predictions_lgbm = le.inverse_transform(predictions_lgbm)
submission["SUBCLASS"] = predictions_lgbm
submission.to_csv('submission_lgbm_oh.csv', encoding='UTF-8-sig', index=False)